# Modulo 23 Atividade 02
#### João Paulo Costa

## Random Forest

#### 1) Passo a passo do algoritmo Random Forest

O Random Forest (ou Floresta Aleatória) é uma extensão do Bagging, aplicada a árvores de decisão.
Ele adiciona um passo extra de aleatoriedade na escolha dos atributos, o que reduz ainda mais a correlação entre as árvores.

Passos:

  1. Bootstrap (amostragem com reposição):

   * Cria várias amostras aleatórias do conjunto de treino, com reposição (como no Bagging).

  2. Seleção aleatória de variáveis (feature selection):

   * Em cada divisão (nó) de uma árvore, em vez de considerar todas as variáveis preditoras, o algoritmo escolhe um subconjunto aleatório de variáveis.

   * Isso garante que as árvores fiquem mais “diversas”.

  3. Treinamento de várias árvores (modelagem):

   * Cada árvore é treinada em sua amostra bootstrap e usa apenas parte das variáveis disponíveis em cada divisão.

  4. Agregação das previsões:

   * Para classificação → voto da maioria das árvores.

   * Para regressão → média das previsões das árvores.

  5. Resultado final:

   * Um modelo mais robusto, com baixo viés e baixa variância.

#### 2. Explicação com suas palavras

O Random Forest é como um bagging aprimorado, assim como no mesmo, fortalecemos um modelo de aprendizado treinando várias versões diferentes dele em subconjuntos diferentes do mesmo conjunto de dados, mas ao inves de um modelo simples utilizamos arvores de decisões, uma para cada subconjunto, e possuimos uma aleatoriedade e independencia maior graças ao Feature Selection aplicados apra criação dos subconjuntos. Assim, temos no fim a combinação de várias árvores simples para formar um modelo muito mais estável e com menor chance de overfitting do que uma árvore única ou um bagging.

#### 3. Diferença entre Bagging e Random Forest

| Característica            | Bagging                               | Random Forest                                                        |
| ------------------------- | ------------------------------------- | -------------------------------------------------------------------- |
| Tipo de modelo base       | Qualquer (ex: árvore, regressão, SVM) | Árvores de decisão                                                   |
| Seleção de variáveis      | Usa todas as variáveis disponíveis    | Seleciona aleatoriamente um subconjunto de variáveis em cada divisão |
| Diversidade entre modelos | Vem apenas do *bootstrap*             | Vem do *bootstrap* + *seleção aleatória de features*                 |
| Correlação entre modelos  | Maior                                 | Menor (mais independente)                                            |
| Desempenho                | Bom                                   | Melhor (geralmente)                                                  |


#### 4. Implementação em Python

In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from scipy.stats import mode

# 1. Carregar base de dados
iris = load_iris()
X, y = iris.data, iris.target


# 2. Parâmetros
n_trees = 10           # número de árvores
n_samples = X.shape[0]
n_features = X.shape[1]
max_features = int(np.sqrt(n_features))  # regra comum: sqrt(p)
models = []
features_idx = []


# 3. Treinamento (Bootstrap + Feature Selection + Modelagem)
for i in range(n_trees):
    # Bootstrap (amostra aleatória com reposição)
    idx = np.random.choice(range(n_samples), size=n_samples, replace=True)
    X_boot, y_boot = X[idx], y[idx]

    # Seleção aleatória de features
    feat_idx = np.random.choice(range(n_features), size=max_features, replace=False)
    features_idx.append(feat_idx)

    # Treinar árvore usando apenas as features selecionadas
    model = DecisionTreeClassifier(random_state=i)
    model.fit(X_boot[:, feat_idx], y_boot)
    models.append(model)


# 4. Agregação das previsões
preds = []
for model, feat_idx in zip(models, features_idx):
    preds.append(model.predict(X[:, feat_idx]))

preds = np.array(preds)
y_pred_final = mode(preds, axis=0, keepdims=True).mode[0]
y_pred_final = np.ravel(y_pred_final)


# 5. Avaliação
acc = accuracy_score(y, y_pred_final)
print(f"Acurácia final com Random Forest (manual): {acc:.3f}")

Acurácia final com Random Forest (manual): 0.993
